<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Framework

In [ ]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [ ]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [ ]:
# Preisprogonose
beschaffung_kapa = pd.read_csv(f"{folder}/Beschaffungskapazitaeten.csv", sep=";")


In [ ]:
# Preise
produktion_kosten = pd.read_csv(f"{folder}/Produktionskosten.csv", sep=";", decimal=",")


## Indexmengen

In [ ]:
I = produktion_kapa["Werk"].unique().tolist() # Menge der Produktionsstandorte

## Optimierungsmodell

In [ ]:
scip = Model()

## Entscheidungsvariablen

In [ ]:
# Beschaffungsmengen der jeweiligen Vorprodukte (Flasche und Vitaminwasser)
Z={}
for l in L:
  for i in I:
    for p in VP:
      for t in T:
        Z[l,i,p,t] = scip.addVar(vtype="C", name=f"{l},{i},{p},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 360


## Parameter

In [ ]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

## Zielfunktion

### Zielfunktion
Min $K = \sum_{l,i,p \in VP,t} bc_{lp} * Z_{lipt} + \sum_{l,i,p \in VP,t} tbc_{lip} * Z_{lipt} + \sum_{i,p \in FP,t} pc_{ip} * X_{ipt} + \sum_{i,p,t} lc_{p} * B_{ipt} + \sum_{i,t} oc^+_{it} * O_{it} + \sum_{i,j,p,t} dtc_{ijp} * Y_{ijpt}$

In [ ]:
bc.get(('ErpoPlast', 'Vitaminwasser'),0)

0

In [ ]:
# Minimierung der gesamten Kosten
scip.setObjective(
    sum(bc.get((l,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Kosten Beschaffung
    sum(btc.get((l,i,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Transportkosten Beschaffung
    sum(pc[i,p]*X[i,p,t] for i in I for p in FP for t in T) + # Kosten der Produktion mit regulärer Kapazität
    sum(oc[i]*O[i,t] for i in I for t in T) + # Kosten der Produktion mit zusätzlicher Kapazität
    sum(lc[p]*B[i,p,t] for i in I for p in P for t in T) + # Kosten der Lagerung
    sum(dtc[i,j,p]*Y[i,j,p,t] for i in I for j in J for p in FP for t in T), # Transportkosten Distribution
    sense="minimize"
    )

## Nebenbedingungen

**(1) Lieferantenkapazitäten**

$\sum_{i} Z_{lipt} \le bcap_{lpt}$

$∀ l,p \in VP, t$

In [ ]:
for l in L:
  for p in VP:
    for t in T:
      scip.addCons(quicksum(Z[l,i,p,t] for i in I)<= bcap.get((l,p,t),0))

## Berechnung Lösung

In [ ]:
scip.optimize()
print(scip.getStatus())

optimal
